In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
from tqdm import tqdm
import regex as re
from nltk.corpus import stopwords
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, T5ForConditionalGeneration
import torch
from itertools import cycle
import os 
from transformers import pipeline

from rouge_score import rouge_scorer

# For BM25
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/adamwagnerhoegh/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

## Generate retrieval corpus

In [2]:
path_adam = '/Users/adamwagnerhoegh/Documents/Legal data/domsdatabasen.retsinformation_newer.json'
path_asger = "/Users/asgerkromand/Library/CloudStorage/OneDrive-UniversityofCopenhagen/0. SDS/1 deep learning and nlp/ANLPDP_exam/data/domsdatabasen.retsinformation_newer.json"
path_andreas = '' #missing

# Define a function that can cycle through paths the above paths try them out, and yield the path
def path():
    paths = cycle([path_adam, path_asger, path_andreas])
    for path in paths:
        if path != '':
            try:
                with open(path, 'r') as f:
                    data = json.load(f)
                return data
            except:
                pass
        else:
            raise FileNotFoundError('No path to data found')

retsinfo = path()
    
rag_list = []
idx = 0
for lov in tqdm(retsinfo):
    for kapitel in lov['kapitler']:
        lov_navn = lov['shortName']
        for paragraffer in kapitel['paragraffer']:
            temp_paragraf_dict = {}
            temp_paragraf_dict['paragraf_nr'] = paragraffer['nummer']
            temp_paragraf_dict['lovnavn'] = lov_navn
            temp_paragraf_list = []
            for styk in paragraffer['stk']:
                temp_paragraf_list.append(styk['tekst'])
            temp_paragraf_dict['text'] = ' '.join(temp_paragraf_list)
            rag_list.append(temp_paragraf_dict)

with open("rag_list.txt", "w") as file:
    for item in rag_list:
        file.write(f"{item}\n")

100%|██████████| 1637/1637 [00:00<00:00, 10191.46it/s]


## Generate dev set

In [3]:
# load excel files in dev set folder
import os

dev_set_folder = "devset"

dfs = []
for file in os.listdir(dev_set_folder):
    if file.endswith(".xlsx"):
        df = pd.read_excel(os.path.join(dev_set_folder, file))
        dfs.append(df)

# merge all excel
dev_set = pd.concat(dfs, ignore_index=True)

# add csv
rag_batch_1_with_qa = pd.read_csv("devset/rag_batch_1_with_qa.csv", sep=",").iloc[:, 1:].dropna()
rag_batch_1_with_qa.columns = dev_set.columns
dev_set = pd.concat([dev_set, rag_batch_1_with_qa], ignore_index=True)

# output dev set
dev_set.to_csv("devset/dev_set.csv", index=False)

## Vectorize retrieval corpus

### Sparse retrieval TF-IDF

In [4]:
rag_list2 = rag_list

def preprocess(rag_list):
    # extract and preprocess text
    corpus = [item['text'] for item in rag_list]
    corpus = [re.sub('\\s{2,}', ' ', 
                     re.sub('\\W|[0-9]|§', ' ',
                           item.lower())) for item in corpus]

    # remove stopwords
    #nltk.download('punkt')
    stop_words = set(stopwords.words('danish'))
    corpus = [' '.join(word for word in text.split() 
                      if word not in stop_words) for text in tqdm(corpus)]
    
    return corpus

corpus = preprocess(rag_list2)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

100%|██████████| 42593/42593 [00:00<00:00, 114190.09it/s]


### Dense retrieval

In [5]:
## WRITE LATER

## RAG retriever

### Sparse retrieval pipeline for BM25


In [6]:
def sparse_retrieval(question, sparse_matrix, k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """

    # preprocess and vectorize question
    question_processed = [re.sub('\\s{2,}', ' ', 
                               re.sub('\\W|[0-9]|§', ' ',
                                     question.lower()))]
    
    # remove stopwords
    stop_words = set(stopwords.words('danish'))
    question_processed = [' '.join(word for word in text.split() 
                                 if word not in stop_words) for text in question_processed]
    
    question_vector = vectorizer.transform(question_processed)

    # sparse retrieval (cosine similarity)
    sparse_retrieval = sparse_matrix.dot(question_vector.T).toarray()

    # get top k paragraphs
    top_k = np.argsort(sparse_retrieval.flatten())[-k:]

    return top_k

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
top_k = sparse_retrieval(random_question, X)
for i in top_k:
    print(f'{rag_list2[i]["paragraf_nr"]}: {rag_list2[i]["text"]}')

Hvis interesser skal foranstaltninger mod interessekonflikter i alternative investeringsfondes indretning beskytte imod? 

§ 23.: En forvalter af alternative investeringsfonde skal ved indretningen af sin virksomhed organisatorisk såvel som administrativt træffe alle rimelige foranstaltninger med henblik på at identificere, forhindre, styre og overvåge interessekonflikter for at forhindre, at de skader de forvaltede alternative investeringsfondes eller deres investorers interesser. En forvalter skal kunne identificere interessekonflikter, der vil kunne opstå i forbindelse med forvaltningen af alternative investeringsfonde imellem følgende: Forvalteren, herunder dens ledelse, ansatte eller enhver anden person, der direkte eller indirekte er forbundet med forvalteren ved et kontrolforhold, og de forvaltede alternative investeringsfonde eller investorer i sådanne alternative investeringsfonde. Forvalteren, herunder dens ledelse, ansatte eller enhver anden person, der direkte eller indirek

In [7]:
def bm25_retrieval(question, bm25_model, rag_list, k=3):
    """
    Function that takes a question and returns a list of the most relevant paragraphs based on BM25.
    """
    # Preprocess and tokenize the question
    question_processed = re.sub(r'\s{2,}', ' ', 
                                 re.sub(r'\W|[0-9]|§', ' ', question.lower()))
    stop_words = set(stopwords.words('danish'))
    question_tokens = [word for word in question_processed.split() if word not in stop_words]

    # Get BM25 scores for the query
    scores = bm25_model.get_scores(question_tokens)

    # Get the top k results
    top_k_indices = np.argsort(scores)[-k:][::-1]  # Sort scores in descending order

    # Return the top k paragraphs
    return [(rag_list[i]['paragraf_nr'], rag_list[i]['text'], scores[i]) for i in top_k_indices]

# Example Usage
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(f"Question: {random_question}\n")

# Assuming bm25 is the initialized BM25Okapi model
top_k_results = bm25_retrieval(random_question, bm25, rag_list2, k=3)

# Print top-k results
for paragraf_nr, text, score in top_k_results:
    print(f"{paragraf_nr}: {text} (Score: {score:.2f})")

Question: Hvornår kan politiet videregive oplysninger om enkeltpersoners rent private forhold til andre myndigheder?



NameError: name 'bm25' is not defined

### Create embedding corpus

In [8]:

def create_embedding_matrix(pooling, save=True, save_folder=None):
    # initialise model
    bert_tokenizer = AutoTokenizer.from_pretrained("KennethTM/bert-base-uncased-danish")
    bert_model = AutoModel.from_pretrained("KennethTM/bert-base-uncased-danish")

    # define device
    device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

    # move model to device
    bert_model.to(device)

    # create list of embedding vectors to concatenate into a torch tensor
    embeddings = []

    # index to track numer of mistakes
    idx = 0

    for item in tqdm(rag_list):
        # doing a try and except as some paragraphs may exceed the context window of the BERT (I believe)
        try:
            # tokenize texts
            input_ids = bert_tokenizer.encode(item['text'], return_tensors='pt').to(device)
            # run through BERT
            with torch.no_grad():  # disable gradient computation for inference
                outputs = bert_model(input_ids)
            
            # different kinds of pooling
            if pooling == 'cls':
                embedding_vector = outputs.last_hidden_state[:, 0, :]
            elif pooling == 'max':
                embedding_vector = torch.max(outputs, dim=1)[0]
            elif pooling == 'mean':
                embedding_vector = torch.mean(outputs, dim=1)
            else:
                raise ValueError(f"Unknown pooling method: {pooling}")
            
            # add cls-vector to list of embeddings
            embeddings.append(embedding_vector)
        except:
            # if error then count errors with this
            idx += 1

    print(f'{idx} no. of errors')

    # concatenate list into torch tensor
    embeddings_tensor = torch.cat(embeddings, dim=0)

    if save == True:
        # make sure that folder exists
        os.makedirs(save_folder, exist_ok=True)

        # save tensor 
        torch.save(embeddings_tensor, f'{save_folder}/{pooling}_embeddings_tensor.pt')

    return embeddings_tensor



In [9]:
#create_embedding_matrix(pooling='cls')

### Dense retrieval pipeline

In [10]:
bert_tokenizer = AutoTokenizer.from_pretrained("vesteinn/DanskBERT")
bert_model = AutoModel.from_pretrained("vesteinn/DanskBERT")

/Users/adamwagnerhoegh/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaModel were not initialized from the model checkpoint at vesteinn/DanskBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
embeddings_matrix = torch.load('/Users/adamwagnerhoegh/Documents/SODAS/sem3/nlp_itu/cls_embeddings_DanskBERT.pt')

In [12]:
def dense_retrieval(question, k=3):
    """
    Function that takes a question and returns a list of paragraphs that are most relevant to the question
    """
    
    # Encode the input sentence
    input_ids = bert_tokenizer.encode(question, return_tensors="pt")  # Encode and add batch dimension
    # Pass the input through the model
    
    with torch.no_grad():  # disable gradient computation for inference
        outputs = bert_model(input_ids)

    # Extract the CLS token representation
    cls_vector = outputs.last_hidden_state[:, 0, :]  # CLS token is at position 0
    
    # sparse retrieval (cosine similarity)
    dense_retrieval = embeddings_matrix @ torch.transpose(cls_vector, 0, 1)
    
    # get top k paragraphs
    top_k_indices = torch.sort(dense_retrieval, descending=True, dim=0)[1][:k]

    return top_k_indices

# check if it works using a random question from the dev set
random_question = dev_set.iloc[np.random.randint(0, len(dev_set))]['question, str']
print(random_question, '\n')
top_k = dense_retrieval(random_question, k=3)
for i in top_k:
    print(f'{rag_list[i]["text"]}')

Hvornår kommer de i loven for livs-, ulykkes- og sygeforsikring givne regler om begunstigede til anvendelse? 

Hvis en virksomheds tilsvar af skatter og afgifter m.v., der opkræves efter reglerne i denne lov, for en afregningsperiode er negativt, udbetales beløbet til virksomheden. Såfremt angivelsen henholdsvis indberetningen af beløb omfattet af § 2, stk. 1, 4. pkt. er modtaget rettidigt, sker udbetaling efter stk. 1 senest 3 uger efter modtagelsen af angivelsen henholdsvis indberetningen for den pågældende periode. Kan told- og skatteforvaltningen på grund af virksomhedens forhold ikke foretage kontrol af angivelsen eller indberetningen af beløb omfattet af § 2, stk. 1, 4. pkt., afbrydes udbetalingsfristen, indtil virksomhedens forhold ikke længere hindrer kontrol. Beløb, der skulle have været udbetalt efter stk. 1, kan tilbageholdes, såfremt angivelser eller indberetningen af beløb omfattet af § 2, stk. 1, 4. pkt. vedrørende afsluttede afregningsperioder ikke er indgivet. Negative 

### Evaluation

In [ ]:
def evaluate(model_name, retrieval_method, metric, k):
    """
    model_name = 'KennethTM/gpt-neo-1.3B-danish' or 'strombergnlp/dant5-large'
    retrieval_method = 'tf-idf', 'bm25' or 'dense'
    metric = 'bleu', 'rouge' or 'meteor'
    """
    # set device to mps
    device = torch.device("mps") if torch.backends.mps.is_available() else "cpu"

    # load AutoTokenizer for model
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # generate list of answers to fill
    answers = []

    # load neo
    # the loops are made for each model to not waste compute on loading models for each question

    if model_name == 'KennethTM/gpt-neo-1.3B-danish':
        model = AutoModelForCausalLM.from_pretrained("KennethTM/gpt-neo-1.3B-danish").to(device)
    
        for question in tqdm(dev_set['question, str'], desc='Answering questions with neo'):

            if retrieval_method == 'tf-idf':
                # create list of paragraphs by getting indexes for best hits with sparse_retrieval
                paragraphs = [rag_list[i]['text'] for i in sparse_retrieval(question, X, k=3)]
                # join list into long string
                paragraphs = ' '.join(paragraphs)

            elif retrieval_method == 'bm25':
                paragraphs = bm25_retrieval(rag_list=rag_list, k=3)
            
            elif retrieval_method == 'dense_retrieval':
                # create list of paragraphs by getting indexes for best hits with sparse_retrieval
                paragraphs = [rag_list[i]['text'] for i in dense_retrieval(question, k=3)]
                # join list into long string
                paragraphs = ' '.join(paragraphs)

            # assemble all in prompt
            prompt = f'Kontekst: {paragraphs} Spørgsmål: {question} Svar: '

            # tokenize
            input_ids = tokenizer(prompt, return_tensors="pt").to(device)

            # set max_length to no. of tokens in prompt + 100 (the 100 are thus for generation)
            max_length = int(input_ids['input_ids'].size(1)) + 100

            # generate answer with no_grad() to save compute
            with torch.no_grad():
                outputs = model.generate(
                    input_ids,
                    max_length=max_length,
                    pad_token_id=tokenizer.eos_token_id)
            
            # decode the generated answer
            answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

            answers.append(answer)
    

    # load T5
    elif model_name == 'strombergnlp/dant5-large':
        model = T5ForConditionalGeneration.from_pretrained("strombergnlp/dant5-large").to(device)

        for question in tqdm(dev_set['question, str'], desc='Answering questions with T5'):

            if retrieval_method == 'tf-idf':
                # create list of paragraphs by getting indexes for best hits with sparse_retrieval
                paragraphs = [rag_list[i]['text'] for i in sparse_retrieval(question, X, k=3)]
                # join list into long string
                paragraphs = ' '.join(paragraphs)

            elif retrieval_method == 'bm25':
                paragraphs = bm25_retrieval(rag_list=rag_list, k=3)
            
            elif retrieval_method == 'dense_retrieval':
                # create list of paragraphs by getting indexes for best hits with sparse_retrieval
                paragraphs = [rag_list[i]['text'] for i in dense_retrieval(question, k=3)]
                # join list into long string
                paragraphs = ' '.join(paragraphs)

            # assemble all in input
            input_text = f"Spørgsmål: {question} Kontekst: {paragraphs} Svar:"

            # tokenize
            input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)

            # generate answer with no_grad() to save compute
            with torch.no_grad():
                outputs = model.generate(
                    input_ids,
                    max_length=max_length,
                    pad_token_id=tokenizer.eos_token_id)
            
            with torch.no_grad():
                outputs = model.generate(input_ids, max_length=100)

            # Decode and print the generated answer
            answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

            answers.append(answer)
        
    # choosing metric to evaluate answers
    if metric == 'bleu':
        scores = []
        idx = 0

        for pred_answer, true_answer in tqdm(zip(answers, list(dev_set['answer, str'])), desc='Evaluating with bleu'):
            try:
                scores.append(nltk.translate.bleu_score.sentence_bleu([true_answer], pred_answer))
            except:
                print(f'Error when computing bleu-score at index {idx}')
            idx += 1
    
    elif metric == 'meteor':
        scores = []
        idx = 0

        for pred_answer, true_answer in tqdm(zip(answers, list(dev_set['answer, str'])), desc='Evaluating with meteor'):
            try:
                scores.append(nltk.tranlsate.meteor_score([true_answer], pred_answer))
            except:
                print(f'Error when computing meteor-score at index {idx}')
            idx += 1

    elif metric == 'rouge':
        scores = []
        idx = 0

        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

        for pred_answer, true_answer in tqdm(zip(answers, list(dev_set['answer, str'])), desc='Evaluating with rouge'):
            try:
                scores.append(score.score(true_answer, pred_answer)['rouge1'])
            except:
                print(f'Error when computing meteor-score at index {idx}')
            idx += 1

    print(f'{metric}-scores for {model_name} using {retrieval_method}: {np.mean(scores)}')
        